In [5]:
import os 
from langchain_community.document_loaders import PyMuPDFLoader,PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path



In [1]:
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader

def process_all_pdf(pdf_directory):
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # find pdf files recursively 
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    print(f"Found {len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyMuPDFLoader(str(pdf_file))
            documents = loader.load()

            # add source info in metadata 
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")    

        except Exception as e:
            print(f"ERROR: {e}")

    # ✅ moved these lines OUTSIDE the for-loop
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents


# ✅ Process all PDFs in the data directory
all_pdf_documents = process_all_pdf("../data")


f:\RAG pipeline\rag_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Found 13 PDF files to process

Processing: 000351346.pdf
  ✓ Loaded 8 pages

Processing: 12916_2019_Article_1489.pdf
  ✓ Loaded 14 pages

Processing: 1706.03762v7.pdf
  ✓ Loaded 15 pages

Processing: 24-0251.pdf
  ✓ Loaded 8 pages

Processing: jamapediatrics_voss_2019_oi_190007.pdf
  ✓ Loaded 9 pages

Processing: JPR_A_384373 187..203 - jpr-16-187.pdf
  ✓ Loaded 17 pages

Processing: lupus-2021-000581.pdf
  ✓ Loaded 11 pages

Processing: NP2015-407320.pdf
  ✓ Loaded 7 pages

Processing: pneumonia-clinical-studies.pdf
  ✓ Loaded 8 pages

Processing: Qualitative findings of the Kisumu.pdf
  ✓ Loaded 22 pages

Processing: quotation_fpdf.pdf
  ✓ Loaded 1 pages

Processing: RESEARCH_METHODOLOGY_C_R_KOTHARI_(ENG).pdf
  ✓ Loaded 414 pages

Processing: study on the relations of neurophysiology.pdf
  ✓ Loaded 20 pages

Total documents loaded: 554


In [43]:
%pip install pytesseract pdf2image Pillow

  Using cached pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
Using cached pytesseract-0.3.13-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import os
import io

def extract_text_from_scanned_pdf(pdf_path, output_txt_path="extracted_text.txt"):
    """
    Extracts text from a scanned PDF using OCR and saves it to a text file.
    """
    # 1. Convert PDF pages to a list of images
    print(f"Converting PDF '{pdf_path}' to images...")
    # High DPI (e.g., 300) improves OCR accuracy
    pages = convert_from_path(pdf_path, dpi=300) 

    extracted_text = []

    # 2. Perform OCR on each image
    print("Performing OCR on each page...")
    for i, page in enumerate(pages):
        # Use Pytesseract to extract text from the image
        text = pytesseract.image_to_string(page)
        extracted_text.append(f"— — PAGE {i+1} — —\n")
        extracted_text.append(text)

    # 3. Combine and save the text
    full_text = "".join(extracted_text)
    with open(output_txt_path, 'w', encoding='utf-8') as f:
        f.write(full_text)

    print(f"Text extraction complete. Output saved to '{output_txt_path}'")
    return full_text

# Example Usage:
# Replace "your_file.pdf" with the path to your PDF file
pdf_file_path = "F:\RAG pipeline\data\jamapediatrics_voss_2019_oi_190007.pdf" 
extracted_content = extract_text_from_scanned_pdf(pdf_file_path)


<>:36: SyntaxWarning: invalid escape sequence '\R'
<>:36: SyntaxWarning: invalid escape sequence '\R'
C:\Users\Asus\AppData\Local\Temp\ipykernel_4528\167196892.py:36: SyntaxWarning: invalid escape sequence '\R'
  pdf_file_path = "F:\RAG pipeline\data\jamapediatrics_voss_2019_oi_190007.pdf"


Converting PDF 'F:\RAG pipeline\data\jamapediatrics_voss_2019_oi_190007.pdf' to images...
Performing OCR on each page...
Text extraction complete. Output saved to 'extracted_text.txt'


In [41]:
from pypdf import PdfReader

def extract_text_from_pdf(pdf_path):
    # Open the PDF file in read-binary mode
    with open(pdf_path, 'rb') as filehandle:
        # Create a PdfReader object
        reader = PdfReader(filehandle)

        # Get the total number of pages
        num_pages = len(reader.pages)
        print(f"Total pages: {num_pages}")

        # Initialize an empty string to store all text
        full_text = ""

        # Loop through all pages and extract text
        for page_number in range(num_pages):
            page = reader.pages[page_number]
            text = page.extract_text()
            if text:
                full_text += f"\n--- Page {page_number + 1} ---\n"
                full_text += text
        
        return full_text

# Specify the path to your PDF file
pdf_file_path = 'F:\RAG pipeline\data\jamapediatrics_voss_2019_oi_190007.pdf'

# Call the function and print the extracted text
try:
    document_content = extract_text_from_pdf(pdf_file_path)
    print(document_content)
except FileNotFoundError:
    print(f"Error: The file '{pdf_file_path}' was not found.")


<>:27: SyntaxWarning: invalid escape sequence '\R'
<>:27: SyntaxWarning: invalid escape sequence '\R'
C:\Users\Asus\AppData\Local\Temp\ipykernel_19812\3342508917.py:27: SyntaxWarning: invalid escape sequence '\R'
  pdf_file_path = 'F:\RAG pipeline\data\jamapediatrics_voss_2019_oi_190007.pdf'
Ignoring wrong pointing object 15 0 (offset 0)


Total pages: 9

--- Page 1 ---
D e e d b s n e U d W p W a k d C h f h s W k G m s d p u d m s h n m e n p G l o p n u h m f
R n b h W k h y W s h n m h m B g h k c p d m U h s g = t s h r l R o d b s p t l C h r n p c d p
@ Q ] m c n l h y d c B k h m h b ] k S q h ] k
B ] s] khm U n r r ) L R : I d r r d x R b g v ] qsy ) A @ : I d m ] C ] m hd kr ) A R : @ ] qn m J khm d : M hb j G ] a d q) O g C : O d sd q V ] r g hm f sn m ) L R :
P ] m c d d k S ] qhp ) L @ : S g n l ] r M - Q n a hm r n m ) L C ) L O G : L ] m hr g ] C d r ] h) O g C : I d m m hed q L - O g hkkho r ) O g C :
B ] qk E d hm r sd hm ) L C : S d qqx V hm n f q] c ) O g C : C d m m hr O - V ] kk) O g C
G L O N Q S : M B D @ t s hr l a d g ] u hn q ] k s g d q ] o x hr d e e d b s hu d a t s d w o d m r hu d ] m c c he e hb t ks s n ] b b d r r -
V g hkd l n a hkd s d b g m n kn f x z a ] r d c s g d q ] o x b ] m ] kkd u h] s d v ] hs , khr s r ] m c r b ] kd e n q hm b q d ] r hm f
c d l ] m c ) e d v b khm hb ] k s

In [2]:
all_pdf_documents

[Document(metadata={'producer': 'macOS Version 15.6.1 (Build 24G90) Quartz PDFContext', 'creator': 'Zen', 'creationdate': "D:20251009043506Z00'00'", 'source': '..\\data\\000351346.pdf', 'file_path': '..\\data\\000351346.pdf', 'total_pages': 8, 'format': 'PDF 1.4', 'title': 'The SafeBoosC Phase II Randomised Clinical Trial: A Treatment Guideline for Targeted Near-Infrared-Derived Cerebral Tissue Oxygenation versus Standard Treatment in Extremely Preterm Infants - 000351346.pdf', 'author': '', 'subject': '', 'keywords': '', 'moddate': "D:20251009043506Z00'00'", 'trapped': '', 'modDate': "D:20251009043506Z00'00'", 'creationDate': "D:20251009043506Z00'00'", 'page': 0, 'source_file': '000351346.pdf', 'file_type': 'pdf'}, page_content='D,L‘hk j‘qfdq?j‘qfdq-bnl\n\x00PdrdWpbg\x00Aphdehmfr\x00\x00MdnmZsnknfx\x001/02:0/39060z067\x00\x00CNH9\x000/-0048.///240235\x00\x00Sgd\x00RWedAnnrB\x00OgWrd\x00GG\x00PWmcnlhrdc\x00BkhmhbWk\x00SphWk-\x00:\x00SpdWsldms\x00Ethcdkhmd\x00enp\x00SWpfdsdc\x00KdWp Gme

In [3]:
#chunks
def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """split documents into chunks to improve llm performance"""
    text_splitter=RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n","\n"," ",""]
    )
    split_docs=text_splitter.split_documents(documents)
    print(f"split {len(documents)} documents into {len(split_docs)},chunks")

    if split_docs:
        print(f"\nExample chunk :")
        print(f"content :{split_docs[0].page_content[:200]}...")
        print(f"metadata:{split_docs[0].metadata}")
    return split_docs    

In [6]:
chunks=split_documents(all_pdf_documents)

split 554 documents into 2065,chunks

Example chunk :
content :D,L‘hk j‘qfdq?j‘qfdq-bnl
 PdrdWpbg Aphdehmfr  MdnmZsnknfx 1/02:0/39060z067  CNH9 0/-0048.///240235  Sgd RWedAnnrB OgWrd GG PWmcnlhrdc BkhmhbWk SphWk- : SpdWsldms Ethcdkhmd enp SWpfdsdc KdWp GmepWpdc 
...
metadata:{'producer': 'macOS Version 15.6.1 (Build 24G90) Quartz PDFContext', 'creator': 'Zen', 'creationdate': "D:20251009043506Z00'00'", 'source': '..\\data\\000351346.pdf', 'file_path': '..\\data\\000351346.pdf', 'total_pages': 8, 'format': 'PDF 1.4', 'title': 'The SafeBoosC Phase II Randomised Clinical Trial: A Treatment Guideline for Targeted Near-Infrared-Derived Cerebral Tissue Oxygenation versus Standard Treatment in Extremely Preterm Infants - 000351346.pdf', 'author': '', 'subject': '', 'keywords': '', 'moddate': "D:20251009043506Z00'00'", 'trapped': '', 'modDate': "D:20251009043506Z00'00'", 'creationDate': "D:20251009043506Z00'00'", 'page': 0, 'source_file': '000351346.pdf', 'file_type': 'pdf'}


Embedding and vector store

In [7]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


Vector Store


In [9]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 59


In [10]:
#convert the text to embedding 
text=[doc.page_content for doc in chunks]
text


#generate the embedding 
embeddings=embedding_manager.generate_embeddings(text)

#store in a vector store database
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 2065 texts...


Batches: 100%|██████████| 65/65 [01:31<00:00,  1.41s/it]


Generated embeddings with shape: (2065, 384)
Adding 2065 documents to vector store...
Successfully added 2065 documents to vector store
Total documents in collection: 2124


#this is a complete data inggestion pipeline

In [11]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vectorstore: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vectorstore
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [18]:
rag_retriever.retrieve("implementation of who4 symptoms")

Retrieving documents for query: 'implementation of who4 symptoms'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 18.49it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)


[]

In [ ]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
from pathlib import Path

# ✅ Load environment variables
load_dotenv()

api_key = os.getenv("GROQ_API_KEY")

# ✅ Initialize the model
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0.1,
    max_tokens=1000,
    api_key=api_key
)

def rag_sample(query, retriever, llm, top_k=3):
    """
    Retrieve relevant documents and generate an answer using the provided LLM.
    """
    # ✅ Get top-k relevant documents
    results = rag_retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""

    # ✅ Handle empty results gracefully
    if not context:
        return "No relevant documents found."

    # ✅ Build the prompt
    prompt = f"""
    Use the following context to answer the question concisely.
    Context:
    {context}

    Question: {query}
    Answer:
    """

    # ✅ Invoke LLM and return clean response
    try:
        response = llm.invoke(prompt)
        return response.content.strip() if response and response.content else "No answer generated."
    except Exception as e:
        return f"Error while generating answer: {e}"


In [15]:
answer=rag_sample("2.6. Motor cortex excitability measures",rag_retriever,llm)
print(answer)

Retrieving documents for query: '2.6. Motor cortex excitability measures'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.39it/s]


Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
The motor cortex excitability measures used in the study include:

1. Resting Motor Threshold (RMT)
2. Motor Evoked Potential (MEP)
3. Cortical Silent Period (CSP) duration 

These measures were used to assess the excitability of the primary motor cortex (M1) using transcranial magnetic stimulation (TMS).


In [1]:
import json
from typing import List

# Unstructured for document parsing
from unstructured.partition.pdf import partition_pdf
from unstructured.chunking.title import chunk_by_title

# LangChain components
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv()

f:\RAG pipeline\rag_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [36]:
import os
from unstructured.partition.pdf import partition_pdf

def partition_document(directory_path: str):
    """Extract elements from all PDFs in a directory using unstructured"""

    print(f"📂 Scanning directory: {directory_path}")

    if not os.path.isdir(directory_path):
        raise ValueError(f"❌ Provided path is not a directory: {directory_path}")

    pdf_files = [
        os.path.join(directory_path, f)
        for f in os.listdir(directory_path)
        if f.lower().endswith(".pdf")
    ]

    print(f"📄 Found {len(pdf_files)} PDF files")

    all_elements = []

    for idx, pdf_path in enumerate(pdf_files, start=1):
        pdf_name = os.path.basename(pdf_path)
        print(f"\n📄 ({idx}/{len(pdf_files)}) Partitioning: {pdf_name}")

        elements = partition_pdf(
            filename=pdf_path,
            strategy="hi_res",
            infer_table_structure=True,
            extract_image_block_types=["Image"],
            extract_image_block_to_payload=True
        )

        # ✅ ADD METADATA (CRITICAL FOR RAG)
        for el in elements:
            el.metadata.source = pdf_name
            # page_number may be None for some elements, keep safe
            el.metadata.page_number = getattr(el.metadata, "page_number", None)

        print(f"✅ Extracted {len(elements)} elements from {pdf_name}")
        all_elements.extend(elements)

    print(f"\n🎉 Total extracted elements from all PDFs: {len(all_elements)}")
    return all_elements


In [37]:
directory_path = r"F:\RAG pipeline\data"
elements = partition_document(directory_path)


📂 Scanning directory: F:\RAG pipeline\data
📄 Found 1 PDF files

📄 (1/1) Partitioning: jamapediatrics_voss_2019_oi_190007.pdf
✅ Extracted 238 elements from jamapediatrics_voss_2019_oi_190007.pdf

🎉 Total extracted elements from all PDFs: 238


In [23]:
set([str(type(el)) for el in elements])

{"<class 'unstructured.documents.elements.Footer'>",
 "<class 'unstructured.documents.elements.Header'>",
 "<class 'unstructured.documents.elements.Image'>",
 "<class 'unstructured.documents.elements.ListItem'>",
 "<class 'unstructured.documents.elements.NarrativeText'>",
 "<class 'unstructured.documents.elements.Table'>",
 "<class 'unstructured.documents.elements.Text'>",
 "<class 'unstructured.documents.elements.Title'>"}

In [24]:
elements[36].to_dict()

{'type': 'NarrativeText',
 'element_id': '69043b4bce94d9ad9b56f8c19c231a54',
 'text': 'We. developed. a pathophysiologically. oriented, evi- dence-based treatment guideline and assessed the level of recommendation according to the US. Preventive Services Task Force (USPSTF) System from 2001 [29] (Appen- dix). A new. version of the USPSTF system is currently. in use [30] where the wording of the grade C recommenda- tion. is possibly. the. most important. change. with respect. to.the previous version; it reads:.“... against routinely pro- viding X service for Y population; there may be consider- ations that support providing the service in an individual patient’. One of the actions listed in the present treatment guideline (i.e. volume. expansion. with normal saline) has been classified as a grade C recommendation. This, as well as the remaining actions included, is a suggested. but not mandated intervention, and. all of them should be considered individually according to.the patient’s o

In [5]:
# Gather all images
images = [element for element in elements if element.category == 'Image']
print(f"Found {len(images)} images")

images[0].to_dict()

# Use https://codebeautify.org/base64-to-image-converter to view the base64 text

Found 1 images


{'type': 'Image',
 'element_id': 'cc1c872cd176cfb36e55f29e19fe4179',
 'text': 'KARGER',
 'metadata': {'coordinates': {'points': ((np.float64(118.68162361111109),
     np.float64(2012.9556550000002)),
    (np.float64(118.68162361111109), np.float64(2047.626582777778)),
    (np.float64(250.6982611111111), np.float64(2047.626582777778)),
    (np.float64(250.6982611111111), np.float64(2012.9556550000002))),
   'system': 'PixelSpace',
   'layout_width': 1654,
   'layout_height': 2205},
  'last_modified': '2026-01-20T10:46:41',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 1,
  'image_base64': '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/2wBDAQkJCQwLDBgNDRgyIRwhMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjL/wAARCAAjAIQDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0

In [25]:
def create_chunks_by_title(elements):
    """Create intelligent chunks using title-based strategy"""
    print("🔨 Creating smart chunks...")
    
    chunks = chunk_by_title(
        elements, # The parsed PDF elements from previous step
        max_characters=3000, # Hard limit - never exceed 3000 characters per chunk
        new_after_n_chars=2400, # Try to start a new chunk after 2400 characters
        combine_text_under_n_chars=500 # Merge tiny chunks under 500 chars with neighbors
    )
    
    print(f"✅ Created {len(chunks)} chunks")
    return chunks

# Create chunks
chunks = create_chunks_by_title(elements)

🔨 Creating smart chunks...
✅ Created 21 chunks


In [7]:
# View all chunks
# chunks

# All unique types
set([str(type(chunk)) for chunk in chunks])

{"<class 'unstructured.documents.elements.CompositeElement'>"}

In [8]:
# View a single chunk
# chunks[2].to_dict()

# View original elements
chunks[11].metadata.orig_elements[-1].to_dict()
# Note: 4th chunk has the first image + 11th chunk has the first table in the sample PDF

{'type': 'UncategorizedText',
 'element_id': 'a4617b7581e9242feaeae9c462604092',
 'text': "GZ0Z 4890190 G0 UO ysenB Aq Jpd'gpELGEOOO/IPLEETE/LZL/E/P0",
 'metadata': {'coordinates': {'points': ((np.float64(1586.0),
     np.float64(1060.0)),
    (np.float64(1586.0), np.float64(1528.0)),
    (np.float64(1602.0), np.float64(1528.0)),
    (np.float64(1602.0), np.float64(1060.0))),
   'system': 'PixelSpace',
   'layout_width': 1654,
   'layout_height': 2205},
  'last_modified': '2026-01-20T10:46:41',
  'filetype': 'application/pdf',
  'languages': ['eng'],
  'page_number': 5,
  'file_directory': 'F:\\RAG pipeline\\data',
  'filename': '000351346.pdf',
  'parent_id': '47aa63e98e23410cee282d398a1bcb7e'}}

In [9]:
def separate_content_types(chunk):
    """Analyze what types of content are in a chunk"""
    content_data = {
        'text': chunk.text,
        'tables': [],
        'images': [],
        'types': ['text']
    }
    
    # Check for tables and images in original elements
    if hasattr(chunk, 'metadata') and hasattr(chunk.metadata, 'orig_elements'):
        for element in chunk.metadata.orig_elements:
            element_type = type(element).__name__
            
            # Handle tables
            if element_type == 'Table':
                content_data['types'].append('table')
                table_html = getattr(element.metadata, 'text_as_html', element.text)
                content_data['tables'].append(table_html)
            
            # Handle images
            elif element_type == 'Image':
                if hasattr(element, 'metadata') and hasattr(element.metadata, 'image_base64'):
                    content_data['types'].append('image')
                    content_data['images'].append(element.metadata.image_base64)
    
    content_data['types'] = list(set(content_data['types']))
    return content_data

def create_ai_enhanced_summary(text: str, tables: List[str], images: List[str]) -> str:
    """Create AI-enhanced summary for mixed content"""
    
    try:
        # Initialize LLM (needs vision model for images)
        llm = ChatOpenAI(model="gpt-4o", temperature=0)
        
        # Build the text prompt
        prompt_text = f"""You are creating a searchable description for document content retrieval.

        CONTENT TO ANALYZE:
        TEXT CONTENT:
        {text}

        """
        
        # Add tables if present
        if tables:
            prompt_text += "TABLES:\n"
            for i, table in enumerate(tables):
                prompt_text += f"Table {i+1}:\n{table}\n\n"
        
                prompt_text += """
                YOUR TASK:
                Generate a comprehensive, searchable description that covers:

                1. Key facts, numbers, and data points from text and tables
                2. Main topics and concepts discussed  
                3. Questions this content could answer
                4. Visual content analysis (charts, diagrams, patterns in images)
                5. Alternative search terms users might use

                Make it detailed and searchable - prioritize findability over brevity.

                SEARCHABLE DESCRIPTION:"""

        # Build message content starting with text
        message_content = [{"type": "text", "text": prompt_text}]
        
        # Add images to the message
        for image_base64 in images:
            message_content.append({
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}
            })
        
        # Send to AI and get response
        message = HumanMessage(content=message_content)
        response = llm.invoke([message])
        
        return response.content
        
    except Exception as e:
        print(f"     ❌ AI summary failed: {e}")
        # Fallback to simple summary
        summary = f"{text[:300]}..."
        if tables:
            summary += f" [Contains {len(tables)} table(s)]"
        if images:
            summary += f" [Contains {len(images)} image(s)]"
        return summary

def summarise_chunks(chunks):
    """Process all chunks with AI Summaries"""
    print("🧠 Processing chunks with AI Summaries...")
    
    langchain_documents = []
    total_chunks = len(chunks)
    
    for i, chunk in enumerate(chunks):
        current_chunk = i + 1
        print(f"   Processing chunk {current_chunk}/{total_chunks}")
        
        # Analyze chunk content
        content_data = separate_content_types(chunk)
        
        # Debug prints
        print(f"     Types found: {content_data['types']}")
        print(f"     Tables: {len(content_data['tables'])}, Images: {len(content_data['images'])}")
        
        # Create AI-enhanced summary if chunk has tables/images
        if content_data['tables'] or content_data['images']:
            print(f"     → Creating AI summary for mixed content...")
            try:
                enhanced_content = create_ai_enhanced_summary(
                    content_data['text'],
                    content_data['tables'], 
                    content_data['images']
                )
                print(f"     → AI summary created successfully")
                print(f"     → Enhanced content preview: {enhanced_content[:200]}...")
            except Exception as e:
                print(f"     ❌ AI summary failed: {e}")
                enhanced_content = content_data['text']
        else:
            print(f"     → Using raw text (no tables/images)")
            enhanced_content = content_data['text']
        
        # Create LangChain Document with rich metadata
        doc = Document(
            page_content=enhanced_content,
            metadata={
                "original_content": json.dumps({
                    "raw_text": content_data['text'],
                    "tables_html": content_data['tables'],
                    "images_base64": content_data['images']
                })
            }
        )
        
        langchain_documents.append(doc)
    
    print(f"✅ Processed {len(langchain_documents)} chunks")
    return langchain_documents


# Process chunks with AI
processed_chunks = summarise_chunks(chunks)

🧠 Processing chunks with AI Summaries...
   Processing chunk 1/21
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 2/21
     Types found: ['text', 'image']
     Tables: 0, Images: 1
     → Creating AI summary for mixed content...
     ❌ AI summary failed: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
     → AI summary created successfully
     → Enhanced content preview: Key Words

Near-infrared spectroscopy.: Newborn - Practice guideline - Cardiovascular agents - Hypotension - Oxygen consumption... Carbon dioxide - Hyperoxia

Abstract

Near-infrared. spectroscopy-der...
   Processing chunk 3/21
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 4/21
     Types found: ['text']
     Tables: 0, Images: 0
     → Using raw text (no tables/images)
   Processing chunk 5/21
     Typ

In [10]:
processed_chunks

[Document(metadata={'original_content': '{"raw_text": "Research Briefings\\n\\nNeonatology\\n\\nNeonatology 2013;104:171-178 DOI: 10.1159/000351346.\\n\\nReceived: February 4,.2013. Accepted after revision: April 12, 2013 Published online: August 1, 2013\\n\\nThe SafeBoosC Phase II Randomised Clinical Trial: A Treatment Guideline for Targeted Near-Infrared- Derived Cerebral Tissue Oxygenation versus Standard Treatment in Extremely Preterm Infants\\n\\nb Adelina Pellicer? Gorm Greisen! Manon Benders\\u2018... Olivier Claris4.. Eugene Dempsey\\u00ae Monica Fumagallif Christian Gluud 9 Cornelia Hagmann \\" Lena Hellstrom-Westas | Simon Hyttel-Sorensen\\u00ae.. Petra Lemmers< ..Gunnar Naulaers)... Gerhard Pichler\\u201c Claudia Roll! Topun Austin? ..and the SafeBoosC Trial Group. Frank van BelS Wim van Oeveren\\u2122 Maria Skoog2 Martin Wolf\\u201d\\n\\nDepartment of Neonatology, La Paz University Hospital, Madrid, Spain; \\u00bbDepartment of Neonatology, Rigshospitalet, Copenhagen Univers

In [13]:
from langchain_chroma import Chroma
def export_chunks_to_json(chunks, filename="chunks_export.json"):
    """Export processed chunks to clean JSON format"""
    export_data = []
    
    for i, doc in enumerate(chunks):
        chunk_data = {
            "chunk_id": i + 1,
            "enhanced_content": doc.page_content,
            "metadata": {
                "original_content": json.loads(doc.metadata.get("original_content", "{}"))
            }
        }
        export_data.append(chunk_data)
    
    # Save to file
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(export_data, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Exported {len(export_data)} chunks to {filename}")
    return export_data

# Export your chunks
json_data = export_chunks_to_json(processed_chunks)

✅ Exported 21 chunks to chunks_export.json


In [16]:
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

def create_vector_store(documents, persist_directory="dbv1/chroma_db"):
    """Create and persist ChromaDB vector store"""
    print("🔮 Creating embeddings and storing in ChromaDB...")

    embedding_model = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    print("--- Creating vector store ---")
    vectorstore = Chroma.from_documents(
        documents=documents,
        embedding=embedding_model,
        persist_directory=persist_directory,
        collection_metadata={"hnsw:space": "cosine"}
    )
    print("--- Finished creating vector store ---")

    print(f"✅ Vector store created and saved to {persist_directory}")
    return vectorstore

# Create the vector store
db = create_vector_store(processed_chunks)


🔮 Creating embeddings and storing in ChromaDB...


C:\Users\Asus\AppData\Local\Temp\ipykernel_19812\3170752640.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


--- Creating vector store ---
--- Finished creating vector store ---
✅ Vector store created and saved to dbv1/chroma_db


In [17]:
# After your retrieval
query = "intervention that affects the oxygen delivery to the brain ? "
retriever = db.as_retriever(search_kwargs={"k": 3})
chunks = retriever.invoke(query)

# Export to JSON
export_chunks_to_json(chunks, "rag_results.json")

✅ Exported 3 chunks to rag_results.json


[{'chunk_id': 1,
  'enhanced_content': 'Interventions That Affect Oxygen Delivery to the Brain\n\nIn drawing up the treatment guideline we used. the premise that hypoxia and hyperoxia are risk factors for.an adverse outcome in. the preterm infant and should be avoided. [10-15]. In addition. to. NIRS assessment of the cerebral circulation, a number of other techniques, par-\n\n172\n\nNeonatology.2013;104:171-178 DOI:.10.1159/000351346\n\nticularly echocardiography, are used to assess the neona-\n\ntal transitional circulation and related outcomes [1, 16- 27]. For the purposes of the treatment guideline, a sys- tematic. review. of the literature was. carried. out. focusing on. clinical studies. conducted on. human. neonates. that used NIRS-derived variables as outcome measures sup- porting the. statements listed. in the treatment guideline. for intervention in the SafeBoosC phase II randomised clinical trial. The. literature review. was. conducted. in March 2011 using the database PubMed

In [26]:
def run_complete_ingestion_pipeline(pdf_path: str):
    """Run the complete RAG ingestion pipeline"""
    print("🚀 Starting RAG Ingestion Pipeline")
    print("=" * 50)
    
    # Step 1: Partition
    elements = partition_document(pdf_path)
    
    # Step 2: Chunk
    chunks = create_chunks_by_title(elements)
    
    # Step 3: AI Summarisation
    summarised_chunks = summarise_chunks(chunks)
    
    # Step 4: Vector Store
    db = create_vector_store(summarised_chunks, persist_directory="dbv2/chroma_db")
    
    print("🎉 Pipeline completed successfully!")
    return db

# Run the complete pipeline


In [28]:
db = run_complete_ingestion_pipeline(r"F:\RAG pipeline\data")

🚀 Starting RAG Ingestion Pipeline
📂 Scanning directory: F:\RAG pipeline\data
📄 Found 10 PDF files

📄 (1/10) Partitioning: 000351346.pdf
✅ Extracted 206 elements from 000351346.pdf

📄 (2/10) Partitioning: 12916_2019_Article_1489.pdf
✅ Extracted 252 elements from 12916_2019_Article_1489.pdf

📄 (3/10) Partitioning: 24-0251.pdf
✅ Extracted 141 elements from 24-0251.pdf

📄 (4/10) Partitioning: jamapediatrics_voss_2019_oi_190007.pdf
✅ Extracted 238 elements from jamapediatrics_voss_2019_oi_190007.pdf

📄 (5/10) Partitioning: JPR_A_384373 187..203 - jpr-16-187.pdf
✅ Extracted 295 elements from JPR_A_384373 187..203 - jpr-16-187.pdf

📄 (6/10) Partitioning: lupus-2021-000581.pdf
✅ Extracted 193 elements from lupus-2021-000581.pdf

📄 (7/10) Partitioning: NP2015-407320.pdf
✅ Extracted 106 elements from NP2015-407320.pdf

📄 (8/10) Partitioning: pneumonia-clinical-studies.pdf
✅ Extracted 148 elements from pneumonia-clinical-studies.pdf

📄 (9/10) Partitioning: Qualitative findings of the Kisumu.pdf
✅

In [35]:
# Query the vector store
from langchain_groq import ChatGroq
import os
query = "objective of jama pediatrics? "

retriever = db.as_retriever(search_kwargs={"k": 3})
chunks = retriever.invoke(query)

def generate_final_answer(chunks, query):
    """Generate final answer using multimodal content"""
    
    try:
        # Initialize LLM (needs vision model for images)
        llm = ChatGroq(model="llama-3.3-70b-versatile"
        
        , temperature=0)
        
        # Build the text prompt
        prompt_text = f"""Based on the following documents, please answer this question: {query}

CONTENT TO ANALYZE:
"""
        
        for i, chunk in enumerate(chunks):
            prompt_text += f"--- Document {i+1} ---\n"
            
            if "original_content" in chunk.metadata:
                original_data = json.loads(chunk.metadata["original_content"])
                
                # Add raw text
                raw_text = original_data.get("raw_text", "")
                if raw_text:
                    prompt_text += f"TEXT:\n{raw_text}\n\n"
                
                # Add tables as HTML
                tables_html = original_data.get("tables_html", [])
                if tables_html:
                    prompt_text += "TABLES:\n"
                    for j, table in enumerate(tables_html):
                        prompt_text += f"Table {j+1}:\n{table}\n\n"
            
            prompt_text += "\n"
        
        prompt_text += """
Please provide a clear, comprehensive answer using the text, tables, and images above. If the documents don't contain sufficient information to answer the question, say "I don't have enough information to answer that question based on the provided documents."

ANSWER:"""

        # Build message content starting with text
        message_content = [{"type": "text", "text": prompt_text}]
        
        # Add all images from all chunks
        for chunk in chunks:
            if "original_content" in chunk.metadata:
                original_data = json.loads(chunk.metadata["original_content"])
                images_base64 = original_data.get("images_base64", [])
                
                for image_base64 in images_base64:
                    message_content.append({
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}
                    })
        
        # Send to AI and get response
        message = HumanMessage(content=message_content)
        response = llm.invoke([message])
        
        return response.content
        
    except Exception as e:
        print(f"❌ Answer generation failed: {e}")
        return "Sorry, I encountered an error while generating the answer."

# Usage
final_answer = generate_final_answer(chunks, query)
print(final_answer)

The objective of JAMA Pediatrics is not explicitly stated in the provided documents. However, based on the content, it appears that JAMA Pediatrics is a peer-reviewed medical journal that publishes original research and studies related to pediatric health, including autism spectrum disorder, socialization, and other child health topics. The journal seems to aim to provide a platform for sharing research findings, discussing clinical implications, and promoting evidence-based practices in pediatric care.

If I had to infer the objective of JAMA Pediatrics based on the provided documents, I would say that its objective is to:

* Publish original research and studies on various aspects of pediatric health
* Provide a forum for discussion and debate on clinical and research topics related to child health
* Promote evidence-based practices and improve patient outcomes in pediatric care
* Serve as a resource for healthcare professionals, researchers, and policymakers working in the field of 